In [ ]:
import torch
from torch import nn
import numpy as np
import sklearn
from sklearn.datasets import make_circles
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split

import torch_directml
import torch.optim as optim

from sklearn.metrics import accuracy_score


In [ ]:
X,y=make_circles(1000,noise=0.03,random_state=42)

len(X),len(y)

In [ ]:
df=pd.DataFrame({"x1":X[:,0], "x2":X[:,1],"y":y})
df

In [ ]:
sns.scatterplot(x=X[:,0],y=X[:,1],hue=y)
plt.xlabel("x1")
plt.ylabel("x2")
plt.show()

In [ ]:
type(X),X.dtype

In [ ]:
#   Here we are changing type from float 64 of numpy to float 32 of torch default type 
X=torch.from_numpy(X).type(torch.float)
y=torch.from_numpy(y).type(torch.float)

In [ ]:
X_train,X_test,y_train,y_test=train_test_split(X,y,random_state=42,test_size=0.2)

len(X_train), len(X_test),len(y_train),len(y_test)

In [ ]:
device=torch.device("cuda" if torch.cuda.is_available() else "cpu")

dml=torch_directml.device()

### Notes

1. **Loss Calculation**: The loss should be calculated using the raw outputs from the model, not the rounded predictions.
2. **Accuracy Calculation**: The accuracy should be calculated using the rounded predictions.
3. **Model Initialization**: The model should be initialized before defining the optimizer.
/

In [ ]:
# Calculate accuracy (a classification metric)
def accuracy_fn(y_true, y_pred):
    correct = torch.eq(y_true, y_pred).sum().item() # torch.eq() calculates where two tensors are equal
    acc = (correct / len(y_pred)) * 100 
    return acc

In [ ]:
class CircleModelV0(nn.Module):
    def __init__(self):
        super().__init__()
        # 2. Create 2 nn.Linear layers capable of handling X and y input and output shapes
        self.layer_1 = nn.Linear(in_features=2, out_features=5) # takes in 2 features (X), produces 5 features
        self.layer_2 = nn.Linear(in_features=5, out_features=1) # takes in 5 features, produces 1 feature (y)
    
    # 3. Define a forward method containing the forward pass computation
    def forward(self, x):
        # Return the output of layer_2, a single feature, the same shape as y
        return self.layer_2(self.layer_1(x)) # computation goes through layer_1 first then the output of layer_1 goes through layer_2

# 4. Create an instance of the model and send it to target device
model_0 = CircleModelV0()
model_0

# Create a loss function
# loss_fn = nn.BCELoss() # BCELoss = no sigmoid built-in
loss_fn = nn.BCEWithLogitsLoss() # BCEWithLogitsLoss = sigmoid built-in

# Create an optimizer
optimizer = torch.optim.SGD(params=model_0.parameters(), 
                            lr=0.1)


torch.manual_seed(42)

# Set the number of epochs
epochs = 1000

# Put data to target device
X_train, y_train = X_train.to(device), y_train
X_test, y_test = X_test.to(device), y_test

# Build training and evaluation loop
for epoch in range(epochs):
    ### Training
    model_0.train()

    # 1. Forward pass (model outputs raw logits)
    y_logits = model_0(X_train).squeeze() # squeeze to remove extra `1` dimensions, this won't work unless model and data are on same device 
    y_pred = torch.round(torch.sigmoid(y_logits)) # turn logits -> pred probs -> pred labls
  
    # 2. Calculate loss/accuracy
    # loss = loss_fn(torch.sigmoid(y_logits), # Using nn.BCELoss you need torch.sigmoid()
    #                y_train) 
    loss = loss_fn(y_logits, # Using nn.BCEWithLogitsLoss works with raw logits
                   y_train) 
    acc = accuracy_fn(y_true=y_train, 
                      y_pred=y_pred) 

    # 3. Optimizer zero grad
    optimizer.zero_grad()

    # 4. Loss backwards
    loss.backward()

    # 5. Optimizer step
    optimizer.step()

    ### Testing
    model_0.eval()
    with torch.inference_mode():
        # 1. Forward pass
        test_logits = model_0(X_test).squeeze() 
        test_pred = torch.round(torch.sigmoid(test_logits))
        # 2. Caculate loss/accuracy
        test_loss = loss_fn(test_logits,
                            y_test)
        test_acc = accuracy_fn(y_true=y_test,
                               y_pred=test_pred)

    # Print out what's happening every 10 epochs
    if epoch % 10 == 0:
        print(f"Epoch: {epoch} | Loss: {loss:.5f}, Accuracy: {acc:.2f}% | Test loss: {test_loss:.5f}, Test acc: {test_acc:.2f}%")

In [ ]:
from helper_functions import plot_predictions, plot_decision_boundary

plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.title("Train")
plot_decision_boundary(model_0, X_train, y_train)
plt.subplot(1, 2, 2)
plt.title("Test")
plot_decision_boundary(model_0, X_test, y_test)

## Coding with non linear model so we can make good classification boundries

In [ ]:
import torch
from torch import nn

import torch.optim as optim
import numpy as np
from sklearn.datasets import make_circles
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

In [ ]:
X,y=make_circles(n_samples=1000,
noise=0.03,
random_state=42
 )

plt.scatter(X[:,0],X[:,1],c=y)

In [ ]:
X_train,X_test,y_train,y_test=train_test_split(X,y,random_state=42, test_size=0.2)

X_train=torch.from_numpy(X_train).type(dtype=torch.float)
X_test=torch.from_numpy(X_test).type(dtype=torch.float)
y_train=torch.from_numpy(y_train).type(dtype=torch.float)
y_test=torch.from_numpy(y_test).type(dtype=torch.float)

In [ ]:
len(X_train),len(X_test),len(y_train),len(y_test)

In [ ]:
X_train[0:3], y_test[0:10]

In [ ]:
nn.ReLU

In [ ]:
#  building model with non linearity


class NonLinearModel(nn.Module):
    def __init__(self):
        super().__init__()

        self.sequential = nn.Sequential(
    nn.Linear(in_features=2, out_features=32),
    nn.ReLU(),
    nn.Linear(32, 64),
    nn.ReLU(),  
    nn.Linear(64, 1)
)

    def forward(self,x):

        return self.sequential(x)


model_3=NonLinearModel()
model_3

In [ ]:
torch.manual_seed(42)

loss_fn=nn.BCEWithLogitsLoss()

optimizer=optim.Adam(params=model_3.parameters(),lr=0.01)

In [ ]:
epochs=20

for epoch in range(epochs):

    model_3.train()

    y_logits=model_3(X_train).squeeze()
    y_pred=torch.round(torch.sigmoid(y_logits))

    loss=loss_fn(y_logits,y_train)

    acc=accuracy_fn(y_train,y_pred)

    optimizer.zero_grad()

    loss.backward()

    optimizer.step()
    model_3.eval()

    with torch.inference_mode():
        #  test loss and pred and dont need to update gradient so normal code

        y_test_logits=model_3(X_test).squeeze()

        y_test_pred=torch.round(torch.sigmoid(y_test_logits))

        test_loss=loss_fn(y_test_logits,y_test)
        test_acc=accuracy_fn(y_test,y_test_pred)

    if epoch%10==0:

        print(f"Epoch: {epoch} | Loss: {loss:.4f} | accu: {acc}% | Test Loss: {test_loss:.4f} | Test_ACC: {test_acc}%")




In [ ]:
from helper_functions import plot_predictions, plot_decision_boundary

plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.title("Train")
plot_decision_boundary(model_3, X_train, y_train)
plt.subplot(1, 2, 2)
plt.title("Test")
plot_decision_boundary(model_3, X_test, y_test)

In [ ]:
acc, test_acc

In [ ]:
model_3(X_train).squeeze().shape